In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
from dotenv import load_dotenv
import pandas as pd
import pydeck as pdk
from IPython.display import display

sys.path.append('../src')

load_dotenv()

from infrastructure.postgres_db import PostgresDB
from application.maps.choropleth_map import ChoroplethMapPlotly

db_name = os.environ.get("DB_NAME")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")


db = PostgresDB(db_name=db_name, db_user=db_user, db_password=db_password)


In [4]:
comunas = db.run_gpd_query("SELECT * FROM comuna")

# Function to extract coordinates
def get_polygon_coordinates(geom):
    if geom.type == 'Polygon':
        return [(x, y) for x, y in geom.exterior.coords]
    elif geom.type == 'MultiPolygon':
        return [[(x, y) for x, y in polygon.exterior.coords] for polygon in geom.geoms][0]
    else:
        return None

comunas = comunas.to_crs(epsg=4326)

comunas = comunas.assign(
    lat = comunas.geom.centroid.y,
    lon = comunas.geom.centroid.x,
    coordinates = comunas.geom.apply(lambda geom: [list(coord) for coord in geom.geoms[0].exterior.coords]),
    coordinates2 = comunas.geom.apply(get_polygon_coordinates),
    score=1000,
)


/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/jm/Code/mapgpt-mvp/.conda/lib/python3.11/site-packages/geopandas/io/sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)
/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykernel_98767/2116503485.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat = comunas.geom.centroid.y,
/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykern

In [ ]:
size_mb = comunas["geom"].simplify(10, preserve_topology=True).memory_usage() / 1024
print(f"The size of the 'comunas' DataFrame is approximately {size_mb:.2f} KB.")

In [ ]:
size_mb = comunas["geom"].memory_usage() / 1024
print(f"The size of the 'comunas' DataFrame is approximately {size_mb:.2f} KB.")

In [7]:
comunas_proj = comunas.to_crs(epsg=3857)
tolerance = 1000  # Adjust the tolerance as needed (100 meters in this example)
comunas_proj.geom = comunas_proj.geom.simplify(tolerance, preserve_topology=True)

comunas_simplified = comunas_proj.to_crs(epsg=4326)


In [ ]:
size_mb = comunas_simplified["geom"].memory_usage(deep=True) / 1024
print(f"The size of the 'comunas' DataFrame is approximately {size_mb:.2f} KB.")

In [ ]:
comunas["geom"].memory_usage(deep=True)

In [ ]:
# comunas.geom.apply(lambda geom: len([list(coord) for coord in geom.geoms[0].exterior.coords])).describe()
# comunas_simplified.geom.apply(get_polygon_coordinates).apply(len).describe()

In [8]:
cm =  ChoroplethMapPlotly(comunas_simplified, "id")
cm.show()

In [44]:
display(cm.fig)

In [ ]:

cm.show()